In [1]:
from dj.sql.parsing.backends.sqloxide import parse, parse_sql, parse_name
from dj.sql.parsing import ast
from dj.construction.inference import get_type_of_expression

SQL = """
SELECT cast(cell_id as int) cell_id,
              cell_name test_cell_name 
            FROM
             (
                 select test_id,
                        cells 
                  from ab_test_detail_d_v2
             ) tests
             CROSS JOIN UNNEST(cells) AS t(cell_id, cell_name)
"""
print(parse_sql(SQL, dialect='ansi'))
parsed = parse(
    SQL,
    dialect="ansi"
)

# Drill down to the UNNEST:
print(parsed)#['Query']['body']['Select']['from'][0]['joins'][0]['relation'])


[{'Query': {'with': None, 'body': {'Select': {'distinct': False, 'top': None, 'projection': [{'ExprWithAlias': {'expr': {'Cast': {'expr': {'Identifier': {'value': 'cell_id', 'quote_style': None}}, 'data_type': {'Int': None}}}, 'alias': {'value': 'cell_id', 'quote_style': None}}}, {'ExprWithAlias': {'expr': {'Identifier': {'value': 'cell_name', 'quote_style': None}}, 'alias': {'value': 'test_cell_name', 'quote_style': None}}}], 'from': [{'relation': {'Derived': {'lateral': False, 'subquery': {'with': None, 'body': {'Select': {'distinct': False, 'top': None, 'projection': [{'UnnamedExpr': {'Identifier': {'value': 'test_id', 'quote_style': None}}}, {'UnnamedExpr': {'Identifier': {'value': 'cells', 'quote_style': None}}}], 'from': [{'relation': {'Table': {'name': [{'value': 'ab_test_detail_d_v2', 'quote_style': None}], 'alias': None, 'args': [], 'with_hints': []}}, 'joins': []}], 'lateral_views': [], 'selection': None, 'group_by': [], 'cluster_by': [], 'distribute_by': [], 'sort_by': [], '

In [7]:
[t.table for t in parsed.find_all(ast.Column)]

[None,
 None,
 None,
 Function(name=Name(name='UNNEST', quote_style=''), namespace=Namespace(names=[]), args=[Column(name=Name(name='cells', quote_style=''), namespace=None, _api_column=False)], distinct=False, over=None, is_tvf=True),
 Function(name=Name(name='UNNEST', quote_style=''), namespace=Namespace(names=[]), args=[Column(name=Name(name='cells', quote_style=''), namespace=None, _api_column=False)], distinct=False, over=None, is_tvf=True),
 None,
 None]

In [8]:
print(parsed)

SELECT  CAST(cell_id AS INT) AS cell_id,
	cell_name AS test_cell_name 
 FROM (SELECT  test_id,
	cells 
 FROM ab_test_detail_d_v2
 
) AS tests
CROSS JOIN UNNEST(cells) AS t(cell_id,cell_name)


In [2]:
get_type_of_expression(parsed.select.from_.joins[0].table.child)

GOT HERE <class 'dj.sql.functions.Unnest'> UNNEST(cells)
expression.table None cells


DJParseException: Cannot resolve type of column cells.

In [2]:
from dj.typing import ColumnType

In [7]:
column_type = ColumnType.Array[ColumnType.Int]
column_type = ColumnType.Map[ColumnType.Str, ColumnType.Int]

In [8]:
column_type.args

('STR', 'INT')

In [12]:
unnested_types = [ColumnType.Int, ColumnType.Str]
ColumnType.Row[unnested_types]


'ROW[INT, STR]'